In [ ]:
!pip install ultralytics onnx
from ultralytics import YOLO

model = YOLO("best.pt")
model.export(format="onnx", opset=12, dynamic=True)


ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
from PIL import Image
import onnxruntime as ort
import numpy as np
import os
import json

image_folder = '/content/drive/MyDrive/image_folder/0_000000'
files = os.listdir(image_folder)
print("Files in folder:", files)

# Încarcă modelul ArcFace
session = ort.InferenceSession("/arcface.onnx", providers=["CPUExecutionProvider"])
print("Model loaded:", session)

def preprocess_face(image_path):
    print("Processing:", image_path)
    img = Image.open(image_path).convert('RGB')
    img = img.resize((112, 112))
    img = np.array(img).astype(np.float32)
    img = (img / 127.5) - 1.0         # normalize [-1,1]
    img = np.expand_dims(img, axis=0) # [1,112,112,3]
    img = np.ascontiguousarray(img)   # important pentru ONNXRuntime
    print("Shape after preprocess:", img.shape)
    return img

def get_face_embedding(image_path):
    input_tensor = preprocess_face(image_path)
    inputs = {session.get_inputs()[0].name: input_tensor.astype(np.float32)}
    print("Running inference...")
    output = session.run(None, inputs)[0][0]    # vector de embedding
    print("Raw embedding length:", len(output))
    norm = np.linalg.norm(output)
    normalized = (output / norm).tolist()
    print("Normalized embedding first 5 values:", normalized[:5])
    return normalized

# Generează embedding-uri pentru toate imaginile
known_faces = {}
for file in os.listdir(image_folder):
    if file.lower().endswith((".jpg", ".png", ".jpeg")):
        name = os.path.splitext(file)[0]
        print("\nProcessing file:", file)
        embedding = get_face_embedding(os.path.join(image_folder, file))
        known_faces[name] = embedding

print("\nAll embeddings generated. Total faces:", len(known_faces))
# Salvează în JSON
output_path = '/content/drive/MyDrive/faces_db.json'  # aici în Drive
with open(output_path, "w") as f:
    json.dump(known_faces, f)
print(f"faces_db.json saved successfully at {output_path}!")



Files in folder: ['IMG_2315.jpeg', 'IMG_2265.jpeg', 'IMG_0457.png', 'F7202E62-1C47-4F01-A371-3C760149D9AA.jpeg', 'CDC55FC0-502A-4194-8BD3-73DA21324395.jpeg', 'IMG_1971.jpeg', 'IMG_2191.jpeg', '748CC81B-B450-4798-8E6D-7C18027F1868.jpeg', 'IMG_2185.jpeg']
Model loaded: <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession object at 0x7d6c66e1a210>

Processing file: IMG_2315.jpeg
Processing: /content/drive/MyDrive/image_folder/0_000000/IMG_2315.jpeg
Shape after preprocess: (1, 112, 112, 3)
Running inference...
Raw embedding length: 512
Normalized embedding first 5 values: [-0.02085748314857483, 0.061057280749082565, 0.004415742587298155, -0.07920988649129868, 0.01136841345578432]

Processing file: IMG_2265.jpeg
Processing: /content/drive/MyDrive/image_folder/0_000000/IMG_2265.jpeg
Shape after preprocess: (1, 112, 112, 3)
Running inference...
Raw embedding length: 512
Normalized embedding first 5 values: [-0.02848365530371666, 0.06429973989725113, -0.00924425758421421, -0.071

In [2]:
!pip install ultralytics onnx
!pip install onnxruntime
!pip install numpy
!pip install Pillow

from PIL import Image
import onnxruntime as ort
import numpy as np
import os
import json

image_folder = '/content/drive/MyDrive/image_folder/0_000000'
files = os.listdir(image_folder)
print("Files in folder:", files)

# Încarcă modelul ArcFace
session = ort.InferenceSession("/content/arcface.onnx", providers=["CPUExecutionProvider"])
print("Model loaded:", session)

def preprocess_face(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((112, 112))
    img = np.array(img).astype(np.float32)
    img = (img / 127.5) - 1.0         # normalize [-1,1]
    img = np.expand_dims(img, axis=0) # [1,112,112,3]
    img = np.ascontiguousarray(img)
    return img

def get_face_embedding(image_path):
    input_tensor = preprocess_face(image_path)
    inputs = {session.get_inputs()[0].name: input_tensor.astype(np.float32)}
    output = session.run(None, inputs)[0][0]    # vector de embedding
    normalized = output / np.linalg.norm(output)
    return normalized.tolist()

# Generează embedding-uri pentru toate imaginile
known_faces = {}

for file in os.listdir(image_folder):
    if file.lower().endswith((".jpg", ".png", ".jpeg")):
        # presupunem că numele persoanei e partea din filename înainte de "_"
        person_name = "Cosmin"
        embedding = get_face_embedding(os.path.join(image_folder, file))

        if person_name not in known_faces:
            known_faces[person_name] = []
        known_faces[person_name].append(embedding)

print(f"\nAll embeddings generated. Total people: {len(known_faces)}")

# Salvează în JSON
output_path = '/content/drive/MyDrive/faces_db.json'
with open(output_path, "w") as f:
    json.dump(known_faces, f)
print(f"faces_db.json saved successfully at {output_path}!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 119.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s eta 0:00:00
Files in folder: ['IMG_2265.JPG', 'Screenshot 2025-08-10 113422.PNG', 'Screenshot 2025-08-10 113301.PNG', 'F7202E62-1C47-4F01-A371-3C760149D9AA.JPG', 'IMG_1791.JPG', 'IMG_1862.JPG', 'IMG_2185.JPG', 'IMG_2191.JPG', '748CC81B-B450-4798-8E6D-7C18027F1868.JPG', 'IMG_2315.JPG', 'IMG_1971.JPG', 'CDC55FC0-502A-4194-8BD3-73DA21324395.JPG', 'IMG_1775.JPG', 'IMG_1517.JPG', 'IMG_0457.png']
Model loaded: <onnxruntime.capi.onnxruntime_inference_collection.InferenceSession object at 0x7b4c17a4ec90>

All embeddings generated. Total peopl